In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.initializers import *
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

import time
from sklearn.model_selection import KFold

In [2]:
# Model params
NAME = f"MLP-at{int(time.time())}"
lr = 0.001
optimizer = Adam(lr=lr)
EPOCHS = 600
BATCH_SIZE = 1054

In [13]:
# Load Data
features = np.loadtxt("./features_nils.txt", delimiter=';')

In [14]:
X = np.array([x[0:16] for x in features])
y = np.array([int(x[16])for x in features])

In [15]:
# Dataset Variables
num_features = 16
num_classes = 3

In [16]:
def create_model(X_train):
    model = Sequential()

    model.add(Dense(units=15, kernel_initializer="uniform", bias_initializer="zeros", input_shape=X_train.shape[1:], activation="relu"))

    model.add(Dense(units=num_classes, kernel_initializer="uniform", bias_initializer="zeros", activation="softmax"))

    model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"])

    return model

In [17]:
tensorboard = TensorBoard(
    log_dir=f"logs/{NAME}",
    histogram_freq=1,
    write_graph=True
    )

filepath = "MLP-{epoch:02d}"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor="val_acc", verbose = 0, save_best_only = True, mode = "max"))

kfold = KFold(n_splits=10, shuffle=True)
fold_no = 1
acc_per_fold = []
loss_per_fold = []

for train, test in kfold.split(X, y):
    # Create model
    model = create_model(X[train])

    print("------------------------------------")
    print(f"Fitting Fold no. {fold_no}")
    model.fit(
        X[train], y[train],
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X[test], y[test]),
        #callbacks=[tensorboard, checkpoint]
    )

    results = model.evaluate(X[test], y[test])
    print(f"Score for {fold_no}: {results}")

    # Push fold loss
    loss_per_fold.append(results[0])

    # Push fold acc
    acc_per_fold.append(results[1])

    fold_no = fold_no + 1

print('--------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('--------------------------------------------------------------')
    print(f"> Fold {i+1} - Accuracy: {acc_per_fold[i]}%")
print('--------------------------------------------------------------')
print("Average scores for all folds:")
print(f"> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})")
print('--------------------------------------------------------------')

------------------------------------
Fitting Fold no. 1
Epoch 1/600
14/14 [==============================] - 0s 35ms/step - loss: nan - accuracy: 0.3556 - val_loss: nan - val_accuracy: 0.3639
Epoch 2/600
14/14 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.3576 - val_loss: nan - val_accuracy: 0.3639
Epoch 3/600
14/14 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.3576 - val_loss: nan - val_accuracy: 0.3639
Epoch 4/600
14/14 [==============================] - 0s 16ms/step - loss: nan - accuracy: 0.3576 - val_loss: nan - val_accuracy: 0.3639
Epoch 5/600
14/14 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.3576 - val_loss: nan - val_accuracy: 0.3639
Epoch 6/600
14/14 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.3576 - val_loss: nan - val_accuracy: 0.3639
Epoch 7/600
14/14 [==============================] - 0s 18ms/step - loss: nan - accuracy: 0.3576 - val_loss: nan - val_accuracy: 0